make figure showing:

- Merapi true colour
- Merapi slope
- Merapi shadow overlay map
- interferogram
- pixel offset range and azimuth


In [1]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [2]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# Indonesia shapefile
IND_COAST = './test_data/DEM/IDN_adm/IDN_adm0.shp' 

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

S1_LON_FILE = './test_data/S1_dsc/geo/20200616.lon'
S1_LAT_FILE = './test_data/S1_dsc/geo/20200616.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# DEM, SLOPE and NDVI file
DEM_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m.tif'
SLOPE_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif'
NDVI_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_NDVI_WGS84_v2.tif'
TRUE_COL_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_true_color_WGS84.tif'
LS_MAP_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/c20200910.ls_map_dem_seg.png'
IFG_FILE1 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20201113_img.diff.gc.tif'
IFG_FILE2 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200926_20201113_img.diff.gc.tif'
IFG_FILE3 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20210812_img.diff.gc.tif'
IFG_FILE4 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20210217_20210218_img.diff.gc.tif'
# PWR_FILE = './test_data/CSK_dsc/rmlis/c20200910.rmli.gc.tif'
PWR_FILE = './test_data/CSK_dsc/rmlis/c20200910.crop.rmli.ras'

S1_IFG_FILE = './test_data/S1_dsc/20200908_20201119/20200908_20201119.diff.gc.tif'
S1_SPOT_FILE = './test_data/S1_dsc/20200908_20201119/disp_20200908_20201119.txt'
S1_SPOT_FILE_CCS = './test_data/S1_dsc/20200908_20201119/20200908_20201119.ccs'
# S1_PWR_FILE = './test_data/S1_dsc/20200908_20201119/20200908.rmli.gc.tif'
S1_PWR_FILE = './test_data/S1_dsc/20200908_20201119/20200908.crop.rmli.ras'

# lava flow files:
L1888_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp'
L1948_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1948.shp'
L1956_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1956.shp'
L1992_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp'
L1997_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1997.shp'
L1998_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp'
CRATER_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp'
# define map region of interest
lon_lims = [110.41, 110.45]
lat_lims = [-7.555, -7.535]



# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN2 = r"^c20200926_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN3 = r"^c20200927_c20210812_disp_[0-9]+_[0-9]+\.txt$"
PATTERN4 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS2 = r"^c20200926_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS3 = r"^c20200927_c20210812_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS4 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"


# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read ls map
ls_map = mplimg.imread(LS_MAP_FILE)

# get world borders via geopandas

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

S1_WIDTH = 24910
S1_LINES = 4185
S1_PWR_WIDTH = 2310
S1_PWR_LINES = 1215
S1_WIDTH_CCS = 262
S1_LINES_CCS = 546
S1_R_START = 2804
S1_A_START = 1743
S1_R_STEP = 10
S1_A_STEP = 2
S1_HEADING = 192.0799295
S1_MEAN_INC = 39.1333

In [3]:
# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
matching_files2 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN2,0)
matching_files_ccs2 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS2,0)
matching_files3 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN3,0)
matching_files_ccs3 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS3,0)
matching_files4 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN4,0)
matching_files_ccs4 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS4,0)


# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)
print(matching_files2)
print(matching_files_ccs2)
print(matching_files3)
print(matching_files_ccs3)
print(matching_files4)
print(matching_files_ccs4)


['c20200927_c20201113_disp_58_28.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_224_108.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_388_188.txt']
['c20200927_c20201113_ccs_58_28', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_224_108', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_388_188']
['c20200926_c20201113_disp_58_28.txt', 'c20200926_c20201113_disp_140_68.txt', 'c20200926_c20201113_disp_224_108.txt', 'c20200926_c20201113_disp_306_148.txt', 'c20200926_c20201113_disp_388_188.txt']
['c20200926_c20201113_ccs_58_28', 'c20200926_c20201113_ccs_140_68', 'c20200926_c20201113_ccs_224_108', 'c20200926_c20201113_ccs_306_148', 'c20200926_c20201113_ccs_388_188']
['c20200927_c20210812_disp_58_28.txt', 'c20200927_c20210812_disp_140_68.txt', 'c20200927_c20210812_disp_224_108.txt', 'c20200927_c20210812_disp_306_148.txt', 'c20200927_c20210812_disp_388_188.txt']
['c20200927_c20210812_ccs_58_28', 'c20200927_c20210812_ccs_1

In [4]:
# load data from files into class multi-kernel
example_pairs = []
for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4]):
    datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                            matching_files,
                                            DIRECTORY_PATH_CCS,
                                            matching_files_ccs,
                                            LAT_FILE,
                                            LON_FILE,
                                            HEADING,
                                            MEAN_INC,
                                            LINES_CCS,
                                            WIDTH_CCS)
    # We need to assign some data not stored in the disp.txt files.
    datastack.get_params_from_file_name()
    datastack.get_latlon_from_file(WIDTH)
    datastack.add_lat_lon_to_data(R_START,A_START)
    datastack.crop_stack_ccs(R_STEP,A_STEP)
    # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
    # Next we add all the offset data (disp.txt) to the stack
    stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
    # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

    # add stack to list
    example_pairs.append(datastack)



In [5]:
## Sentinel-1 offset tracking pair

S1_data = pd.read_csv(S1_SPOT_FILE, header  =None, sep = '\s+')

# get latitude and longitude data from file
S1_lon_vec = np.fromfile(S1_LON_FILE, dtype='>f', count=-1)
S1_lat_vec = np.fromfile(S1_LAT_FILE, dtype='>f', count=-1)
# set 0,0 coordinates to nan
S1_lon_vec[S1_lon_vec == 0] = np.nan
S1_lat_vec[S1_lat_vec == 0] = np.nan
# get number of lines in lat lon files
lines_ll = S1_lon_vec.shape[0]/S1_WIDTH
# reshape tomatrix for index referencing
S1_LON = np.reshape(S1_lon_vec,[int(lines_ll),S1_WIDTH])
S1_LAT = np.reshape(S1_lat_vec,[int(lines_ll),S1_WIDTH]) 

S1_data[9]  = S1_LAT[S1_data[1],S1_data[0]]
S1_data[10] = S1_LON[S1_data[1],S1_data[0]]

print(S1_data)


         0     1      2      3      4      5      6      7       8         9   \
0      3294  1909 -0.100  0.064 -0.121 -0.026  0.404  0.049   1.266 -7.497390   
1      3304  1909 -0.095  0.076 -0.121 -0.026  0.398  0.060   1.425 -7.497278   
2      3314  1909 -0.109  0.079 -0.120 -0.026  0.389  0.027   1.477 -7.497250   
3      3284  1911 -0.117  0.047 -0.121 -0.026  0.412  0.009   1.016 -7.497804   
4      3294  1911 -0.104  0.061 -0.121 -0.026  0.407  0.040   1.214 -7.497621   
...     ...   ...    ...    ...    ...    ...    ...    ...     ...       ...   
66633  4824  2669 -0.900 -0.582 -0.187  0.016  0.138 -1.660  -8.406 -7.580797   
66634  4834  2669  0.428 -0.716 -0.189  0.016  0.127  1.435 -10.277 -7.580739   
66635  4794  2671 -0.303 -0.210 -0.183  0.016  0.181 -0.278  -3.179 -7.581275   
66636  4804  2671 -0.435 -0.375 -0.185  0.016  0.159 -0.582  -5.499 -7.581219   
66637  4814  2671 -0.788 -0.502 -0.186  0.016  0.141 -1.402  -7.280 -7.581149   

               10  
0      

In [6]:
import fiona
from matplotlib.colors import ListedColormap
from matplotlib import cm as mpl_cm
## find nearest elevation, slope, and NDVI values


def read_tiff(filename,n_bands):
    with rio.open(filename) as src:

        values = [src.read(i+1) for i in range(n_bands)]
        if n_bands > 1:
            values_stack = np.stack(values,axis=-1)
        else:
            values_stack = values[0]
        print(f'Data {filename} has shape', values_stack.shape)
        height = values_stack.shape[0]
        width = values_stack.shape[1]
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        xs, ys = rio.transform.xy(src.transform, rows, cols)
        lons = np.array(xs)
        lats = np.array(ys)
        extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]
    return values_stack, lons, lats, extent 


# load indonesia borders
ind_coastlines = gpd.read_file(IND_COAST)


DEM_heights, DEM_lons, DEM_lats, DEM_extent = read_tiff(DEM_FILE,1)
DEM_slope, slope_lons, slope_lats, slope_extent = read_tiff(SLOPE_FILE,1)

## NDVI open and crop data to DEM extent
# define map region of interest
lon_lims = [np.nanmin(DEM_lons), np.nanmax(DEM_lons)]
lat_lims = [np.nanmin(DEM_lats), np.nanmax(DEM_lats)]

# define map region of interest
lon_lims = [110.405, 110.46]
lat_lims = [-7.555, -7.515]

crop_flag=1


# create cropping polygon from ROI
if crop_flag:
    coords = ((lon_lims[0], lat_lims[0]), (lon_lims[0], lat_lims[1]), (lon_lims[1], lat_lims[1]), (lon_lims[1], lat_lims[0]), (lon_lims[0], lat_lims[0]))
    crop_poly = Polygon(coords)
    crop_poly_geojson = gpd.GeoSeries([crop_poly])
    crop_poly_geojson.to_file('./test_data/crop_ndvi_poly_v2.shp',crs="EPSG:4326")


with fiona.open("./test_data/crop_ndvi_poly_v2.shp", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

# with

# get data and read coords from first file
with rio.open(NDVI_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_data/ndvi_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

NDVI, ndvi_lons, ndvi_lats, ndvi_extent = read_tiff("./test_data/ndvi_cropped.tif",1)


with rio.open(TRUE_COL_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/TRUE_COL_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

TRUE_COL, TRUE_COL_lons, TRUE_COL_lats, TRUE_COL_extent = read_tiff("./test_Data/TRUE_COL_cropped.tif",3)
CSK_IFG1, CSK_IFG_lons1, CSK_IFG_lats1, CSK_IFG_extent1 = read_tiff(IFG_FILE1,3)
CSK_IFG2, CSK_IFG_lons2, CSK_IFG_lats2, CSK_IFG_extent2 = read_tiff(IFG_FILE2,3)
CSK_IFG3, CSK_IFG_lons3, CSK_IFG_lats3, CSK_IFG_extent3 = read_tiff(IFG_FILE3,3)
CSK_IFG4, CSK_IFG_lons4, CSK_IFG_lats4, CSK_IFG_extent4 = read_tiff(IFG_FILE4,3)
# CSK_PWR, CSK_PWR_lons, CSK_PWR_lats, CSK_PWR_extent = read_tiff(PWR_FILE,1)
# with rio.open(PWR_FILE) as src:
#     CSK_PWR = src.read(1)

CSK_PWR = plt.imread(PWR_FILE)


S1_IFG, S1_IFG_lons, S1_IFG_lats, S1_IFG_extent = read_tiff(S1_IFG_FILE,3)
# S1_PWR, S1_PWR_lons, S1_PWR_lats, S1_PWR_extent = read_tiff(S1_PWR_FILE,1)
# with rio.open(S1_PWR_FILE) as src:
#     S1_PWR = src.read(1)
S1_PWR = plt.imread(S1_PWR_FILE)

# with rio.open("./test_Data/TRUE_COL_cropped.tif") as src:
#     TRUE_COL_1 = src.read(1)
#     TRUE_COL_2 = src.read(2)
#     TRUE_COL_3 = src.read(3)
#     print('Band1 has shape', TRUE_COL_1.shape)
#     height = NDVI.shape[0]
#     width = NDVI.shape[1]
#     cols, rows = np.meshgrid(np.arange(width), np.arange(height))
#     xs, ys = rio.transform.xy(src.transform, rows, cols)
#     TRUE_COL_lons = np.array(xs)
#     TRUE_COL_lats = np.array(ys)
#     TRUE_COL_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

# TRUE_COL = np.stack((TRUE_COL_1,TRUE_COL_2,TRUE_COL_3),axis=-1)


# with rio.open(IFG_FILE) as src:
#     IFG_1 = src.read(1)
#     IFG_2 = src.read(2)
#     IFG_3 = src.read(3)
#     print('Band1 has shape', IFG_1.shape)
#     height = DEM_heights.shape[0]
#     width = DEM_heights.shape[1]
#     cols, rows = np.meshgrid(np.arange(width), np.arange(height))
#     xs, ys = rio.transform.xy(src.transform, rows, cols)
#     IFG_lons = np.array(xs)
#     IFG_lats = np.array(ys)
#     IFG_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

# IFG = np.stack((IFG_1,IFG_2,IFG_3),axis=-1)

x= mpl_cm.get_cmap('Blues_r', 135)
y= mpl_cm.get_cmap('YlGn', 135)
z = np.vstack((x(range(135)),
                       y(range(135))))
ndvi_cmap = ListedColormap(z, name='BlYlGn')

Data ./test_data/DEM/TDX_Merapi_WGS84_5m.tif has shape (2028, 1672)
Data ./test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif has shape (2028, 1672)
Data ./test_data/ndvi_cropped.tif has shape (75, 102)
Data ./test_Data/TRUE_COL_cropped.tif has shape (442, 608, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20201113_img.diff.gc.tif has shape (10171, 8384, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200926_20201113_img.diff.gc.tif has shape (10171, 8384, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20200927_20210812_img.diff.gc.tif has shape (10171, 8384, 3)
Data /Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/example_ifgs/20210217_20210218_img.diff.gc.tif has shape (10171, 8384, 3)
Data ./test_data/S1_dsc/20200908_20201119/20200908_20201119.diff.gc.tif has shape (10171, 8384, 3)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_11832/3418123507.py:131: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  x= mpl_cm.get_cmap('Blues_r', 135)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_11832/3418123507.py:132: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  y= mpl_cm.get_cmap('YlGn', 135)


In [7]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

from matplotlib.colors import LinearSegmentedColormap
clist = ['#f579cd', '#f67fc6', '#f686bf', '#f68cb9', '#f692b3', '#f698ad',
                     '#f69ea7', '#f6a5a1', '#f6ab9a', '#f6b194', '#f6b78e', '#f6bd88',
                     '#f6c482', '#f6ca7b', '#f6d075', '#f6d66f', '#f6dc69', '#f6e363',
                     '#efe765', '#e5eb6b', '#dbf071', '#d0f477', '#c8f67d', '#c2f684',
                     '#bbf68a', '#b5f690', '#aff696', '#a9f69c', '#a3f6a3', '#9cf6a9',
                     '#96f6af', '#90f6b5', '#8af6bb', '#84f6c2', '#7df6c8', '#77f6ce',
                     '#71f6d4', '#6bf6da', '#65f6e0', '#5ef6e7', '#58f0ed', '#52e8f3',
                     '#4cdbf9', '#7bccf6', '#82c4f6', '#88bdf6', '#8eb7f6', '#94b1f6',
                     '#9aabf6', '#a1a5f6', '#a79ef6', '#ad98f6', '#b392f6', '#b98cf6',
                     '#bf86f6', '#c67ff6', '#cc79f6', '#d273f6', '#d86df6', '#de67f6',
                     '#e561f6', '#e967ec', '#ed6de2', '#f173d7']
colormap = LinearSegmentedColormap.from_list('dismph', clist, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

dismph = colormap(range(255))
dismph2 = np.row_stack((np.array(dismph),np.array(dismph)))
print(dismph2)

colormap = LinearSegmentedColormap.from_list('dismph2', dismph2, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

[[0.96078431 0.4745098  0.80392157 1.        ]
 [0.96175317 0.48032295 0.79713956 1.        ]
 [0.96272203 0.4861361  0.79035755 1.        ]
 ...
 [0.93347174 0.43354095 0.87510957 1.        ]
 [0.93734717 0.43935409 0.86445213 1.        ]
 [0.94122261 0.44516724 0.85379469 1.        ]]


In [8]:
# EDM data
import datetime
dates = ['2020-04-24',
         '2020-05-01',
         '2020-05-08',
         '2020-05-15',
         '2020-05-22',
         '2020-05-29',
         '2020-06-05',
         '2020-06-12',
         '2020-06-19',
         '2020-06-26',
         '2020-07-03',
         '2020-07-10',
         '2020-07-17',
         '2020-07-24',
         '2020-07-31',
         '2020-08-07',
         '2020-08-14',
         '2020-08-21',
         '2020-08-28',
         '2020-09-04',
         '2020-09-11',
         '2020-09-18',
         '2020-09-25',
         '2020-10-02',
         '2020-10-09',
         '2020-10-16',
         '2020-10-23',
         '2020-10-30',
         '2020-11-06',
         '2020-11-13',
         '2020-11-20',
         '2020-11-27',
         '2020-12-04',
         '2020-12-11',
         '2020-12-18',
         '2020-12-25',
         '2021-01-01',
         '2021-01-08',
         '2021-01-15',
         '2021-01-22',
         '2021-01-29',
         '2021-02-05',
         '2021-02-12',
         '2021-02-19',
         '2021-02-26',
         '2021-03-05',
         '2021-03-12',
         '2021-03-19',
         '2021-03-26',
         '2021-04-01',
         '2021-04-09',
         '2021-04-16',
         '2021-04-23',
         '2021-04-30',
         '2021-05-07',
         '2021-05-14',
         '2021-05-21',
         '2021-06-04',
         '2021-06-11',
         '2021-06-18',
         '2021-06-25',
         '2021-07-02',
         '2021-07-09',
         '2021-07-16',
         '2021-07-23',
         '2021-07-30',
         '2021-08-05',
         '2021-08-13',
         '2021-08-20',
         '2021-08-27',
         '2021-09-03',
         '2021-09-10',
         '2021-09-17',
         '2021-09-24',
         '2021-10-01',
         '2021-10-08',
         '2021-10-15',
         '2021-10-22',
         '2021-10-29',
         '2021-11-05',
         '2021-11-12',
         '2021-11-19',
         '2021-11-26',
         '2021-12-03',
         '2021-12-10',
         '2021-12-17',
         '2021-12-24',
         '2021-12-31']

EDM_dates = [datetime.datetime.strptime(date,'%Y-%m-%d') for date in dates]

EDM_rate = [   0,    0,    0,    0,    0,    0,    0,    0,   -1, -2/7,
            -2/7, -2/7, -1/7,    0, -2/7, -2/7, -1/7, -1/7, -2/7, -2/7,
            -2/7, -2/7, -3/7, -2/7,   -1,   -2,   -4,   -9,  -10,   -9,
             -11,  -11,   -9,   -9,  -11,  -14,  -15,   -6, -0.8, -0.4,
               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
               0,    0, -0.5, -0.6, -0.7,    0,   -1, -1.8, -0.7,   -2,
            -0.7,   -4,   -8,   -7,  -11,  -12, -7.7, -1.9,    0,    0,
               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            -0.2, -0.2, -0.7, -0.5, -0.4, -0.2, -0.2, -0.4]

EDM_dist_min_max = np.array([[4044.834, 4044.840, 3859.108, 3859.115],
                             [4044.836, 4044.841, 3859.106, 3859.112],
                             [4044.830, 4044.840, 3859.102, 3859.114],
                             [4044.832, 4044.837, 3859.106, 3859.113],
                             [4044.836, 4044.842, 3859.108, 3859.114],
                             [4044.836, 4044.841, 3859.110, 3859.116],
                             [4044.834, 4044.840, 3859.110, 3859.116],
                             [4044.835, 4044.846, 3859.110, 3859.115],
                             [4044.769, 4044.841, 3859.055, 3859.122],
                             [4044.738, 4044.763, 3859.027, 3859.051],
                             [4044.719, 4044.738, 3859.011, 3859.031],
                             [4044.697, 4044.718, 3858.996, 3859.017],
                             [4044.690, 4044.701, 3858.995, 3859.003],
                             [4044.681, 4044.691, 3858.990, 3858.998],
                             [4044.659, 4044.682, 3858.976, 3858.990],
                             [4044.640, 4044.657, 3858.962, 3858.974],
                             [4044.626, 4044.641, 3858.952, 3858.963],
                             [4044.604, 4044.621, 3858.934, 3858.948],
                             [4044.580, 4044.598, 3858.925, 3858.932],
                             [4044.558, 4044.574, 3858.905, 3858.922],
                             [4044.540, 4044.556, 3858.902, 3858.908],
                             [4044.510, 4044.526, 3858.875, 3858.893],
                             [4044.466, 4044.500, 3858.849, 3858.874],
                             [4044.433, 4044.459, 3858.828, 3858.844],
                             [4044.319, 4044.338, 3858.759, 3858.772],
                             [4044.183, 4044.319, 3858.663, 3858.759],
                             [4043.795, 4044.117, 3858.347, 3858.639],
                             [4043.102, 4043.780, 3857.719, 3858.334],
                             [4042.361, 4043.057, 3856.911, 3857.681],
                             [4041.388, 4042.227, 3856.099, 3856.899],
                             [4040.604, 4041.383, 3855.342, 3856.090],
                             [4039.829, 4040.567, 3854.566, 3855.308],
                             [4039.179, 4039.801, 3853.850, 3854.537],
                             [4038.727, 4039.087, 3853.442, 3853.816],
                             [4037.761, 4038.238, 3852.457, 3852.954],
                             [4036.760, 4037.753, 3851.429, 3852.448],
                             [4035.710, 4036.748, 3850.333, 3851.419],
                             [4035.318, 4035.707, 3849.916, 3850.328],
                             [4035.243, 4035.297, 3849.843, 3850.899],
                             [4035.220, 4035.245, 3849.812, 3849.837],
                             [4035.213, 4035.218, 3849.804, 3849.811],
                             [4035.164, 4035.214, 3849.763, 3849.802],
                             [4035.158, 4035.168, 3849.748, 3849.763],
                             [4035.148, 4035.160, 3849.741, 3849.758],
                             [4035.146, 4035.156, 3849.742, 3849.753],
                             [4035.145, 4035.153, 3849.738, 3849.749],
                             [4035.131, 4035.148, 3849.729, 3849.746],
                             [4035.124, 4035.137, 3849.724, 3849.735],
                             [4035.093, 4035.124, 3849.705, 3849.725],
                             [4035.091, 4035.098, 3849.700, 3849.709],
                             [4035.083, 4035.097, 3849.695, 3849.708],
                             [4035.059, 4035.090, 3849.678, 3849.702],
                             [4035.021, 4035.058, 3849.651, 3849.681],
                             [4034.976, 4035.022, 3849.616, 3849.653],
                             [4034.910, 4034.973, 3849.557, 3849.616],
                             [4034.885, 4034.903, 3849.539, 3849.554],
                             [4034.820, 4034.887, 3849.485, 3849.542],
                             [4034.623, 4034.742, 3849.311, 3849.421],
                             [4033.827, 4033.874, 3848.518, 3848.558],
                             [4034.441, 4034.572, 3849.143, 3849.267],
                             [4034.403, 4034.452, 3849.105, 3849.154],
                             [4034.084, 4034.385, 3848.797, 3849.087],
                             [4033.528, 4034.078, 3848.231, 3848.793],
                             [4033.039, 4033.522, 3847.721, 3848.222],
                             [4032.321, 4033.033, 3846.975, 3847.716],
                             [4031.519, 4032.220, 3846.140, 3846.867],
                             [4030.990, 4031.499, 3845.602, 3846.119],
                             [4030.780, 4030.845, 3845.388, 3845.529],
                             [4030.770, 4030.785, 3845.375, 3845.390], 
                             [4030.751, 4030.772, 3845.362, 3845.377],
                             [4030.740, 4030.761, 3845.353, 3845.365],
                             [4030.723, 4030.746, 3845.338, 3845.356],
                             [4030.716, 4030.726, 3845.332, 3845.343],
                             [4030.698, 4030.698, 3845.314, 3845.322],
                             [4030.694, 4030.700, 3845.309, 3845.315],
                             [4030.681, 4030.696, 3845.299, 3845.310],
                             [4030.675, 4030.685, 3845.295, 3845.304],
                             [4030.658, 4030.676, 3845.285, 3845.296],
                             [4030.663, 4030.667, 3845.269, 3845.286],
                             [  np.nan,   np.nan, 3845.265, 3845.276],
                             [4030.632, 4030.643, 3845.243, 3845.258],
                             [4030.627, 4030.633, 3845.226, 3845.242],
                             [4030.603, 4030.618, 3845.226, 3845.242],
                             [4030.570, 4030.583, 3845.133, 3845.158],
                             [4030.537, 4030.553, 3845.097, 3845.116],
                             [4030.520, 4030.536, 3845.080, 3845.095],
                             [4030.511, 4030.526, 3845.075, 3845.085],
                             [4030.483, 4030.511, 3845.057, 3845.079]])


dates_lavadomes = ['2020-02-19',
                   '2020-06-13',
                   '2020-07-11',
                   '2020-07-26',
                   '2020-10-29',
                   '2020-11-03',
                   '2020-11-16',
                   '2021-01-04',
                   '2021-01-14',
                   '2021-01-21',  
                   '2021-01-25',
                   '2021-01-28',
                   '2021-02-04',
                   '2021-02-11',
                   '2021-02-17',
                   '2021-02-25',
                   '2021-03-05',
                   '2021-03-11',
                   '2021-03-18',
                   '2021-03-25',
                   '2021-04-01',
                   '2021-04-07',
                   '2021-04-08',
                   '2021-04-15',
                   '2021-04-21',
                   '2021-04-22',
                   '2021-04-28',
                   '2021-05-02',
                   '2021-05-16',
                   '2021-05-20',
                   '2021-05-28',
                   '2021-06-08',
                   '2021-06-17',
                   '2021-06-24',
                   '2021-07-01',
                   '2021-07-08',
                   '2021-07-15',
                   '2021-07-22',
                   '2021-07-28',
                   '2021-08-05',
                   '2021-08-19',
                   '2021-08-26',
                   '2021-09-01',
                   '2021-09-09',
                   '2021-09-16',
                   '2021-09-23',
                   '2021-09-30',
                   '2021-10-07',
                   '2021-10-14',
                   '2021-10-21',
                   '2021-10-28',
                   '2021-11-04',
                   '2021-11-11',
                   '2021-11-18',
                   '2021-11-25',
                   '2021-12-02',
                   '2021-12-09',
                   '2021-12-16',
                   '2021-12-23',
                   '2021-12-30',
]

lavadome_dates = [datetime.datetime.strptime(date,'%Y-%m-%d') for date in dates_lavadomes]


lava_dome_vol_rate =np.array([[291000, np.nan, np.nan, np.nan],
                    [200000, np.nan, np.nan, np.nan],
                    [200000, np.nan, np.nan, np.nan],
                    [200000, np.nan, np.nan, np.nan],
                    [200000, np.nan, np.nan, np.nan],
                    [200000, np.nan, np.nan, np.nan],
                    [200000, np.nan, np.nan, np.nan],
                    [200000, np.nan, 0 , np.nan],
                    [np.nan, np.nan, 46766, 8500],
                    [np.nan, np.nan, 104000,8600],
                    [np.nan, np.nan, 157000, np.nan],
                    [np.nan, np.nan, 62000, np.nan],
                    [np.nan, np.nan, 117400, 12600],
                    [np.nan, np.nan, 295000, 48900],
                    [426000,  10000, 397500, 25200],
                    [np.nan, np.nan, 618700, 13600],
                    [np.nan, np.nan, 711000, 13900],
                    [np.nan, np.nan, 785600, 13500],
                    [950000,  12800, 840000, 12900],
                    [np.nan, np.nan, 949000, 13300],
                    [np.nan, np.nan, 1060000, 13400],
                    [np.nan, np.nan, 1098000, 12800],
                    [1681000, np.nan, np.nan, np.nan],
                    [np.nan, np.nan, 1024800, 12200],
                    [np.nan, np.nan, 1069600, 11900],
                    [1794000, np.nan, np.nan, np.nan],
                    [1794000, np.nan, 1141850, 11900],
                    [1700000, 14000, 1100000, 17000],
                    [np.nan, np.nan, 1160000, 11500],
                    [np.nan, np.nan, 1284000, 11700],
                    [np.nan, np.nan, 1264800, 11600],
                    [2100000, np.nan, 1300000, np.nan],
                    [np.nan, np.nan, 1390000, 11300],
                    [np.nan, np.nan, 1590000, 11400],
                    [np.nan, np.nan, 1680000, 11800],
                    [2741000, np.nan, 1815000, np.nan],
                    [2796000, np.nan, 1830000, np.nan],
                    [2808000, np.nan, 1880000, np.nan],
                    [2817000, np.nan, 1878000, np.nan],
                    [np.nan, np.nan, 1895000, np.nan],
                    [np.nan, np.nan, 1350000, np.nan],
                    [2831000, np.nan, 1400000, np.nan],
                    [2842000, np.nan, 1440000, np.nan],
                    [2854000, np.nan, 1550000, np.nan],
                    [2854000, np.nan, 1550000, np.nan],
                    [2854000, np.nan, 1600000, np.nan],
                    [2854000, np.nan, 1630000, np.nan],
                    [2854000, np.nan, 1679000, np.nan],
                    [2927000, np.nan, 1609000, np.nan],
                    [2927000, np.nan, 1609000, np.nan],
                    [2927000, np.nan, 1609000, np.nan],
                    [2927000, np.nan, 1610000, np.nan],
                    [2927000, np.nan, 1610000, np.nan],
                    [2927000, np.nan, 1610000, np.nan],
                    [2927000, np.nan, 1610000, np.nan],
                    [2948000, np.nan, 1615000, np.nan],
                    [3007000, np.nan, 1629000, np.nan],
                    [3007000, np.nan, 1622000, np.nan],
                    [3007000, np.nan, 1654000, np.nan],
                    [3007000, np.nan, 1630000, np.nan]
                   ])

# June 21, 2020 19000 m^3 eroded from southwest sector of Lava-1997
# Oct. 18, 2020 too foggy to get volume measurement
# Nov. 16, 2020 morphology changes due to collapse of old lava ofr lava 1997, 1998, 1888, and 1954
# Dec. 5, 2020 nonew lava dome
# Dec. 14, 2020 no new lava dome
# On January 4, 2021 at 19.50 WIB, the first incandescent lava avalanche was recorded on a seismogram with an amplitude of 33 mm and a duration of 60 seconds.
# Aug 12, reported that SW lava dome volume is 'relatively fixed'

In [9]:
%matplotlib osx
## overview figure

# define path to shapefile

# Read the shapefile
L1888 = gpd.read_file(L1888_FILE)
L1948 = gpd.read_file(L1948_FILE)
L1956 = gpd.read_file(L1956_FILE)
L1992 = gpd.read_file(L1992_FILE)
L1997 = gpd.read_file(L1997_FILE)
L1998 = gpd.read_file(L1998_FILE)
CRATER = gpd.read_file(CRATER_FILE)

# Extract latitude and longitude into separate columns
coords_L1888 = np.array(list(L1888["geometry"][0].coords))
coords_L1956_1 = np.array(list(L1956["geometry"][0].coords))
coords_L1956_2 = np.array(list(L1956["geometry"][1].coords))
coords_L1948 = np.array(list(L1948["geometry"][0].coords))
coords_L1992 = np.array(list(L1992["geometry"][0].coords))
coords_L1997 = np.array(list(L1997["geometry"][0].coords))
coords_L1998 = np.array(list(L1998["geometry"][0].coords))
coords_CRATER = np.array(list(CRATER["geometry"][0].coords))


In [10]:

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
# define map region of interest
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

CSK_obj1 = example_pairs[0].Stack[1]
CSK_obj2 = example_pairs[1].Stack[1]
CSK_obj3 = example_pairs[2].Stack[1]
CSK_obj4 = example_pairs[3].Stack[1]

merapi_fig, merapi_ax = plt.subplots(2,1,height_ratios=[10,5])

# true colour
merapi_ax[0].imshow(TRUE_COL,extent=TRUE_COL_extent)
merapi_ax[0].scatter(x=[110.410626],y=[-7.526175],s=50,marker='s',color='white',edgecolor='black' , label= 'Pos Badaban')
# merapi_ax.scatter(x=[110.439757],y=[-7.537567],s=50,marker='v',color='tab:orange',edgecolor='black', label='EDM reflector')
merapi_ax[0].scatter(x=[110.440165],y=[-7.537451],s=50,marker='v',color='tab:orange',edgecolor='black', label='EDM reflectors')
# merapi_ax.scatter(x=[110.439260],y=[-7.537533],s=20,marker='v',color='tab:orange',edgecolor='black', label='EDM reflector')
merapi_ax[0].plot([lon_lims[0],lon_lims[0],lon_lims[1],lon_lims[1],lon_lims[0]],[lat_lims[0],lat_lims[1],lat_lims[1],lat_lims[0],lat_lims[0]],linewidth=2,color='red')
merapi_ax[0].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
merapi_ax[0].plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
merapi_ax[0].plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
merapi_ax[0].plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
merapi_ax[0].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
merapi_ax[0].plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
merapi_ax[0].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
merapi_ax[0].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
merapi_ax[0].plot([110.410626,110.440165],[-7.526175,-7.537451],linewidth=2,color='black',linestyle='-.',label='EDM sightline')
merapi_ax[0].text(lon_lims[0]+0.006,lat_lims[0]+0.001,'Figure 7 outline',color='red',weight='bold')


merapi_ax_1 = merapi_ax[0].inset_axes([0.5,0.75,0.5,0.3])
merapi_ax_1.add_patch(Rectangle((90,-15),55,25,color='white'))
ind_coastlines.plot(ax=merapi_ax_1,color='lightgrey',edgecolor='black')
merapi_ax_1.scatter(110.4,-7.54,s=20,c='red',marker='^',label='Merapi')
merapi_ax_1.text(106.7,-7.2,'Java',weight='bold')
merapi_ax_1.set_aspect('equal','box')
merapi_ax_1.set_xlim((105,117))
merapi_ax_1.set_ylim((-9,-5))
merapi_ax_1.legend(loc=3,frameon=False,handletextpad=0.1,borderaxespad=0.01)
merapi_ax_1.set_axis_off()

merapi_ax[0].set_aspect('equal','box')
merapi_ax[0].set_xlim([110.41, 110.46])
merapi_ax[0].set_ylim(-7.555, -7.515)
merapi_ax[0].add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
    "family": "serif",
    "size": "large",
}))
merapi_ax[0].legend(loc=3)
merapi_ax[0].set_axis_off()

start_week=0

for week_id in enumerate(EDM_dates[start_week:-1]):
    if not np.isnan(EDM_dist_min_max[start_week + week_id[0],0]):
        merapi_ax[1].fill_between(x= [EDM_dates[start_week + week_id[0]],EDM_dates[start_week + week_id[0]+1]],
                     y1 = EDM_dist_min_max[start_week + week_id[0],0]-EDM_dist_min_max[start_week,0],
                     y2 = EDM_dist_min_max[start_week + week_id[0],1]-EDM_dist_min_max[start_week,1],
                     color = 'tab:blue',
                     alpha = 0.3)
        merapi_ax[1].fill_between(x= [EDM_dates[start_week + week_id[0]],EDM_dates[start_week + week_id[0]+1]],
                     y1 = EDM_dist_min_max[start_week + week_id[0],2]-EDM_dist_min_max[start_week,2],
                     y2 = EDM_dist_min_max[start_week + week_id[0],3]-EDM_dist_min_max[start_week,3],
                     color = 'tab:orange',
                     alpha = 0.3)
merapi_ax[1].set_ylabel('EDM distance change [m]')
merapi_ax_twin = merapi_ax[1].twinx()
merapi_ax_twin.set_ylabel('EDM change rate [cm/day]', color = 'red') 
merapi_ax_twin.hlines(y=EDM_rate[:-1], xmin=EDM_dates[:-1], xmax=EDM_dates[1:],
              color='red', zorder=-1)
merapi_ax_twin.vlines(x=EDM_dates[1:], ymin=EDM_rate[:-1], ymax=EDM_rate[1:], color='red',
              linestyle='dashed', zorder=-1)

merapi_ax_twin.spines['right'].set_color('red')
merapi_ax_twin.yaxis.label.set_color('red')
merapi_ax_twin.tick_params(axis='y', colors='red')


merapi_ax[1].fill_between(x= [],
                     y1 = [],
                     y2 = [],
                     color = 'tab:blue',
                     alpha = 0.3,
                     label = 'EDM range BAD-RB1 \n[x+'+ str(EDM_dist_min_max[start_week,0]) + ' m]')
merapi_ax[1].fill_between(x= [],
                     y1 = [],
                     y2 = [],
                     color = 'tab:orange',
                     alpha = 0.3,
                     label = 'EDM range BAD-RB2 \n[x+' + str(EDM_dist_min_max[start_week,2]) + ' m]')
merapi_ax[1].hlines(y=[], xmin=[], xmax=[],
              color='red', zorder=1, label = 'Reported EDM rate change [cm/day]')

merapi_ax[1].legend(loc='lower left',framealpha=0.3)

merapi_ax[1].set_xticklabels(merapi_ax[1].get_xticklabels(), rotation=45, ha='right')

plt.subplots_adjust(hspace=0.1)

/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_46845/1133256711.py:103: UserWarning: FixedFormatter should only be used together with FixedLocator
  merapi_ax[1].set_xticklabels(merapi_ax[1].get_xticklabels(), rotation=45, ha='right')


In [10]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

In [11]:
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# overview map 2:

# | true col | NDVI | S1 amp | CSK amp |
# |           EDM            | legend  |

# define map region of interest
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]

fig = plt.figure(figsize=(10,8))

gs = GridSpec(3, 2, figure=fig,height_ratios=[10,10,7])

ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax01 = fig.add_subplot(gs[0, 1]) # Merapi NDVI
ax02 = fig.add_subplot(gs[1, 0]) # S1 amplitude (with summit inset?)
ax03 = fig.add_subplot(gs[1, 1]) # CSK amplitude (with summit inset?)

ax10 = fig.add_subplot(gs[2, 0:2]) # EDM time series


# true colour

ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00.scatter(x=[110.410626],y=[-7.526175],s=50,marker='s',color='white',edgecolor='black' , label= 'Pos Badaban')
# merapi_ax.scatter(x=[110.439757],y=[-7.537567],s=50,marker='v',color='tab:orange',edgecolor='black', label='EDM reflector')
ax00.scatter(x=[110.440165],y=[-7.537451],s=50,marker='v',color='tab:orange',edgecolor='black', label='EDM reflectors')
# merapi_ax.scatter(x=[110.439260],y=[-7.537533],s=20,marker='v',color='tab:orange',edgecolor='black', label='EDM reflector')
ax00.plot([lon_lims[0],lon_lims[0],lon_lims[1],lon_lims[1],lon_lims[0]],[lat_lims[0],lat_lims[1],lat_lims[1],lat_lims[0],lat_lims[0]],linewidth=2,color='red')
ax00.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax00.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
ax00.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
ax00.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
ax00.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
ax00.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
ax00.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax00.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax00.plot([110.410626,110.440165],[-7.526175,-7.537451],linewidth=2,color='black',linestyle='-.',label='EDM sightline')
ax00.text(lon_lims[0]+0.012,lat_lims[0]+0.001,'Figure 7 outline',color='red',weight='bold')


merapi_ax_1 = ax00.inset_axes([0.5,0.75,0.5,0.3])
merapi_ax_1.add_patch(Rectangle((90,-15),55,25,color='white'))
ind_coastlines.plot(ax=merapi_ax_1,color='lightgrey',edgecolor='black')
merapi_ax_1.scatter(110.4,-7.54,s=20,c='red',marker='^',label='Merapi')
merapi_ax_1.text(106.7,-7.2,'Java',weight='bold')
merapi_ax_1.set_aspect('equal','box')
merapi_ax_1.set_xlim((105,117))
merapi_ax_1.set_ylim((-9,-5))
merapi_ax_1.legend(loc=3,frameon=False,handletextpad=0.01,borderaxespad=0.0,borderpad=0.1)
merapi_ax_1.set_axis_off()

ax00.set_aspect('equal','box')
ax00.set_xlim([110.41, 110.46])
ax00.set_ylim(-7.555, -7.515)
ax00.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
    "family": "serif",
    "size": "large",
}))
# ax00.legend(loc=3,labelspacing=0.1)
ax00.legend(bbox_to_anchor =(-0.6,0), loc='lower left',labelspacing=0.1)



# ndvi with contours
ndvi_plot = ax01.imshow(NDVI,cmap=ndvi_cmap,vmin=-1,vmax=1,extent=ndvi_extent)
ax01.contour(np.flipud(NDVI),levels=[-0.1,0.1,0.5],extent=ndvi_extent,colors='k',linewidth=2)
cax = add_right_cax(ax01, 0.005, 0.02)
cbar = fig.colorbar(ndvi_plot, cax=cax,label='NDVI [-]')
# S1 pwr
ax02.imshow(S1_PWR[470:675,1075:1335],cmap='Greys')

# CSK pwr
ax03.imshow(CSK_PWR[3400:5710,1440:3770],cmap='Greys')
ax03.set_aspect(0.75)

# EDM line

start_week=0

for week_id in enumerate(EDM_dates[start_week:-1]):
    if not np.isnan(EDM_dist_min_max[start_week + week_id[0],0]):
        ax10.fill_between(x= [EDM_dates[start_week + week_id[0]],EDM_dates[start_week + week_id[0]+1]],
                     y1 = EDM_dist_min_max[start_week + week_id[0],0]-EDM_dist_min_max[start_week,0],
                     y2 = EDM_dist_min_max[start_week + week_id[0],1]-EDM_dist_min_max[start_week,1],
                     color = 'tab:blue',
                     alpha = 0.3)
        ax10.fill_between(x= [EDM_dates[start_week + week_id[0]],EDM_dates[start_week + week_id[0]+1]],
                     y1 = EDM_dist_min_max[start_week + week_id[0],2]-EDM_dist_min_max[start_week,2],
                     y2 = EDM_dist_min_max[start_week + week_id[0],3]-EDM_dist_min_max[start_week,3],
                     color = 'tab:orange',
                     alpha = 0.3)
ax10.set_ylabel('EDM distance change [m]')
merapi_ax_twin = ax10.twinx()
merapi_ax_twin.set_ylabel('EDM change rate [cm/day]', color = 'red') 
merapi_ax_twin.hlines(y=EDM_rate[:-1], xmin=EDM_dates[:-1], xmax=EDM_dates[1:],
              color='red', zorder=-1)
merapi_ax_twin.vlines(x=EDM_dates[1:], ymin=EDM_rate[:-1], ymax=EDM_rate[1:], color='red',
              linestyle='dashed', zorder=-1)

merapi_ax_twin.spines['right'].set_color('red')
merapi_ax_twin.yaxis.label.set_color('red')
merapi_ax_twin.tick_params(axis='y', colors='red')


ax10.fill_between(x= [],
                     y1 = [],
                     y2 = [],
                     color = 'tab:blue',
                     alpha = 0.3,
                     label = 'EDM range BAD-RB1 \n[x+'+ str(EDM_dist_min_max[start_week,0]) + ' m]')
ax10.fill_between(x= [],
                     y1 = [],
                     y2 = [],
                     color = 'tab:orange',
                     alpha = 0.3,
                     label = 'EDM range BAD-RB2 \n[x+' + str(EDM_dist_min_max[start_week,2]) + ' m]')
ax10.hlines(y=[], xmin=[], xmax=[],
              color='red', zorder=1, label = 'Reported EDM rate change [cm/day]')

ax10.axvline(x=datetime.datetime.strptime('2021-01-04','%Y-%m-%d'),color='k',linestyle='--',linewidth=2)

ax10.legend(loc='lower left',framealpha=0.3)

ax10.set_xticklabels(ax10.get_xticklabels(), rotation=45, ha='right')

# turn axes off
ax00.set_axis_off()
ax01.set_axis_off()
ax02.set_axis_off()
ax03.set_axis_off()

fig.subplots_adjust(hspace=0.01,wspace=0.01)
fig.savefig('/Users/markbemelmans/Documents/PhD/projects/Merapi2021/figures/Merapi_overview_5panel.pdf',dpi=300)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_11832/1306071162.py:69: UserWarning: The following kwargs were not used by contour: 'linewidth'
  ax01.contour(np.flipud(NDVI),levels=[-0.1,0.1,0.5],extent=ndvi_extent,colors='k',linewidth=2)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_11832/1306071162.py:127: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax10.set_xticklabels(ax10.get_xticklabels(), rotation=45, ha='right')


In [12]:
plt.close('all')

In [13]:
%matplotlib osx
## overview figure

# 1   4   7  10  13 16 
# 2   5   8  11  14 17
# 3   6   9  12  15 18

# 1: True colour S2
# 2: CSK Amplitude
# 3: S1 Amplitude
# 4: S1 ifg
# 5: S1 range off
# 6: S1 azi off
# 7,10,13,16: CSK ifg
# 8,11,14,17: CSK range off
# 9,12,15,18: CSK azi off

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
# define map region of interest
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]

# zoom_lon_lims = [110.428, 110.45]
# zoom_lat_lims = [-7.545, -7.5305]
orientation = 'vertical'

zoom_lon_lims = [110.4387, 110.4495]
zoom_lat_lims = [-7.5411, -7.5344]

CSK_obj1 = example_pairs[0].Stack[1]
CSK_obj2 = example_pairs[1].Stack[1]
CSK_obj3 = example_pairs[2].Stack[1]
CSK_obj4 = example_pairs[3].Stack[1]


fig = plt.figure(figsize=(9,10))

gs = GridSpec(6, 5, figure=fig, height_ratios=[10,8,10,8,10,8])
# ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
# ax20 = fig.add_subplot(gs[2, 0]) # CSK amplitude (with summit inset?)
# ax40 = fig.add_subplot(gs[4, 0]) # S1 amplitude (with summit inset?)

ax01 = fig.add_subplot(gs[0, 0]) # S1 ifg
ax11 = fig.add_subplot(gs[1, 0]) # S1 ifg zoom
ax21 = fig.add_subplot(gs[2, 0]) # S1 range off
ax31 = fig.add_subplot(gs[3, 0]) # S1 range off zoom
ax41 = fig.add_subplot(gs[4, 0]) # S1 azi off
ax51 = fig.add_subplot(gs[5, 0]) # S1 azi off zoom

ax02 = fig.add_subplot(gs[0, 1]) # CSK1 ifg
ax12 = fig.add_subplot(gs[1, 1]) # CSK1 ifg zoom
ax22 = fig.add_subplot(gs[2, 1]) # CSK1 range off
ax32 = fig.add_subplot(gs[3, 1]) # CSK1 range off zoom
ax42 = fig.add_subplot(gs[4, 1]) # CSK1 azi off
ax52 = fig.add_subplot(gs[5, 1]) # CSK1 azi off

ax03 = fig.add_subplot(gs[0, 2]) # CSK2 ifg
ax13 = fig.add_subplot(gs[1, 2]) # CSK2 ifg zoom
ax23 = fig.add_subplot(gs[2, 2]) # CSK2 range off
ax33 = fig.add_subplot(gs[3, 2]) # CSK2 range off zoom
ax43 = fig.add_subplot(gs[4, 2]) # CSK2 azi off
ax53 = fig.add_subplot(gs[5, 2]) # CSK2 azi off zoom

ax04 = fig.add_subplot(gs[0, 3]) # CSK3 ifg
ax14 = fig.add_subplot(gs[1, 3]) # CSK3 ifg zoom
ax24 = fig.add_subplot(gs[2, 3]) # CSK3 range off
ax34 = fig.add_subplot(gs[3, 3]) # CSK3 range off zoom
ax44 = fig.add_subplot(gs[4, 3]) # CSK3 azi off
ax54 = fig.add_subplot(gs[5, 3]) # CSK3 azi off zoom

ax05 = fig.add_subplot(gs[0, 4]) # CSK4 ifg
ax15 = fig.add_subplot(gs[1, 4]) # CSK4 ifg zoom
ax25 = fig.add_subplot(gs[2, 4]) # CSK4 range off
ax35 = fig.add_subplot(gs[3, 4]) # CSK4 range off zoom
ax45 = fig.add_subplot(gs[4, 4]) # CSK4 azi off
ax55 = fig.add_subplot(gs[5, 4]) # CSK4 azi off zoom


# true colour
# ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
# ax00.set_xlim(lon_lims)
# ax00.set_ylim(lat_lims)



# ax00.scatter(x=[110.410626],y=[-7.526175],s=20,marker='s',color='tab:blue')
# ax00.scatter(x=[110.439757],y=[-7.537567],s=20,marker='v',color='tab:orange')
# ax00.scatter(x=[110.440165],y=[-7.537451],s=20,marker='v',color='tab:orange')
# ax00.scatter(x=[110.439260],y=[-7.537533],s=20,marker='v',color='tab:orange')

# ax00_1 = ax00.inset_axes([0.4,0.77,0.6,0.3])
# ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
# # world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
# ind_coastlines.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
# ax00_1.scatter(110.4,-7.54,s=5,c='red')
# ax00_1.set_aspect('equal','box')
# ax00_1.set_xlim((105,117))
# ax00_1.set_ylim((-9,-5))
# ax00_1.set_axis_off()

# # S1 pwr
# ax20.imshow(S1_PWR[470:675,1075:1335],cmap='Greys')

# # CSK pwr
# ax40.imshow(CSK_PWR[3400:5710,1440:3770],cmap='Greys')

# S1 ifg
ax01.imshow(S1_IFG,extent=S1_IFG_extent)
ax01.set_xlim(lon_lims)
ax01.set_ylim(lat_lims)
ax11.imshow(S1_IFG,extent=S1_IFG_extent)
ax11.set_xlim(zoom_lon_lims)
ax11.set_ylim(zoom_lat_lims)

# S1 r off
S1_r_off_map = ax21.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax21.set_xlim(lon_lims)
ax21.set_ylim(lat_lims)
S1_r_off_map_zoom = ax31.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax31.set_xlim(zoom_lon_lims)
ax31.set_ylim(zoom_lat_lims)

# S1 a off
S1_a_off_map = ax41.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax41.set_xlim(lon_lims)
ax41.set_ylim(lat_lims)
S1_a_off_map_zoom = ax51.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax51.set_xlim(zoom_lon_lims)
ax51.set_ylim(zoom_lat_lims)

# CSK ifg 1
ax02.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax02.set_xlim(lon_lims)
ax02.set_ylim(lat_lims)
ax12.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax12.set_xlim(zoom_lon_lims)
ax12.set_ylim(zoom_lat_lims)

# CSK r off 1
CSK_r_off_map = ax22.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax22.set_xlim(lon_lims)
ax22.set_ylim(lat_lims)
CSK_r_off_map_zoom = ax32.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax32.set_xlim(zoom_lon_lims)
ax32.set_ylim(zoom_lat_lims)
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 1
CSK_a_off_map = ax42.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
CSK_a_off_map_zoom = ax52.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 2
ax03.imshow(CSK_IFG2,extent=CSK_IFG_extent2)
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
ax13.imshow(CSK_IFG2,extent=CSK_IFG_extent2)
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)

# CSK r off 2
CSK_r_off_map = ax23.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
CSK_r_off_map = ax33.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 2
CSK_a_off_map = ax43.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
CSK_a_off_map = ax53.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 3
ax04.imshow(CSK_IFG3,extent=CSK_IFG_extent3)
ax04.set_xlim(lon_lims)
ax04.set_ylim(lat_lims)
ax14.imshow(CSK_IFG3,extent=CSK_IFG_extent3)
ax14.set_xlim(zoom_lon_lims)
ax14.set_ylim(zoom_lat_lims)

# CSK r off 3
CSK_r_off_map = ax24.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax24.set_xlim(lon_lims)
ax24.set_ylim(lat_lims)
CSK_r_off_map = ax34.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax34.set_xlim(zoom_lon_lims)
ax34.set_ylim(zoom_lat_lims)

# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 3
CSK_a_off_map = ax44.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax44.set_xlim(lon_lims)
ax44.set_ylim(lat_lims)
CSK_a_off_map = ax54.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax54.set_xlim(zoom_lon_lims)
ax54.set_ylim(zoom_lat_lims)

# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 4
ax05.imshow(CSK_IFG4,extent=CSK_IFG_extent4)
ax05.set_xlim(lon_lims)
ax05.set_ylim(lat_lims)
ax15.imshow(CSK_IFG4,extent=CSK_IFG_extent4)
ax15.set_xlim(zoom_lon_lims)
ax15.set_ylim(zoom_lat_lims)

# CSK r off 4
CSK_r_off_map = ax25.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax25.set_xlim(lon_lims)
ax25.set_ylim(lat_lims)
CSK_r_off_map = ax35.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax35.set_xlim(zoom_lon_lims)
ax35.set_ylim(zoom_lat_lims)
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 4
CSK_a_off_map = ax45.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax45.set_xlim(lon_lims)
ax45.set_ylim(lat_lims)
CSK_a_off_map = ax55.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax55.set_xlim(zoom_lon_lims)
ax55.set_ylim(zoom_lat_lims)
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)

for a in [ax00,ax01,ax21,ax41,ax02,ax22,ax42,ax03,ax23,ax43,ax04,ax24,ax44,ax05,ax25,ax45]:
    a.set_aspect('equal','box')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.plot([zoom_lon_lims[0],zoom_lon_lims[1],zoom_lon_lims[1],zoom_lon_lims[0],zoom_lon_lims[0]],
           [zoom_lat_lims[0],zoom_lat_lims[0],zoom_lat_lims[1],zoom_lat_lims[1],zoom_lat_lims[0]],
           linewidth=2,color='red')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
        "family": "serif",
        "size": "large",
    }))

for a in [ax11,ax31,ax51,ax12,ax32,ax52,ax13,ax33,ax53,ax14,ax34,ax54,ax15,ax35,ax55]:
    a.set_aspect('equal','box')
    
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    # a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
    #     "family": "serif",
    #     "size": "large",
    # }))
    a.set_xlim(zoom_lon_lims)
    a.set_ylim(zoom_lat_lims)
    a.set_axis_off()
    

for (a,s) in zip([ax01,ax21,ax41,ax02,ax22,ax42,ax03,ax23,ax43,ax04,ax24,ax44,ax05,ax25,ax45],['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O']):
    a.annotate(s,xy=[lon_lims[0]+0.002,lat_lims[0]+0.002],xytext=[lon_lims[0]+0.002,lat_lims[0]+0.002],xycoords='data',backgroundcolor='white',fontsize=16)
    a.set_axis_off()
# ax20.annotate('B', xy=[0.01,0.01],xytext=[0.01,0.01],xycoords='axes fraction',backgroundcolor='white',fontsize=16)
# ax20.set_axis_off()
# ax40.annotate('C', xy=[0.01,0.01],xytext=[0.01,0.01],xycoords='axes fraction',backgroundcolor='white',fontsize=16)
# ax40.set_axis_off()

ax05_cmap = fig.add_axes([0.9,0.67,0.01,0.33])
ax05_cmap.set_axis_off()

fake_data = ax05.imshow(np.random.random((10,2)),cmap=colormap, vmin=-1,vmax=1,alpha = 1)



# ax05_cax = add_right_cax(ax05,pad=0.01,width=0.01)
# cbar0 = fig.colorbar(fake_data, cax=ax05_cax, orientation='vertical',)
# ticklabs = cbar0.ax.get_yticklabels()
# cbar0.ax.set_yticklabels(ticklabs, fontsize=16)
# cbar0.set_label( label='LOS change', fontsize=16)
# cbar0.set_ticks(ticks=[-1, 0, 1], labels=[r'-λ', '0', r'+λ'])
# ax15_cax = add_right_cax(ax15,pad=0.01,width=0.01)
# cbar1 = fig.colorbar(CSK_r_off_map, cax=ax15_cax, orientation='vertical', )
# ticklabs = cbar1.ax.get_yticklabels()
# cbar1.ax.set_yticklabels(ticklabs,fontsize=16)
# cbar1.set_label(label='Range offset [m]', size=16)
# cbar1.set_ticks(ticks=[-3, 0, 3],labels=['-3','0','3'])
# ax25_cax = add_right_cax(ax25,pad=0.01,width=0.01)
# cbar2 = fig.colorbar(CSK_a_off_map, cax=ax25_cax, orientation='vertical')
# ticklabs = cbar2.ax.get_yticklabels()
# print(ticklabs)
# cbar2.ax.set_yticklabels(ticklabs,fontsize=16)
# cbar2.set_label(label= 'Azimuth offset [m]',size=16)
# cbar2.set_ticks(ticks=[-3, 0, 3],labels=['-3','0','3'])



# set titles
ax01.set_title('Sentinel-1\n2020-09-08\n2020-11-19\nBperp: 11 m ')
ax02.set_title('CSK Pair 1\n2020-09-27\n2020-11-13\nBperp: 4 m')
ax03.set_title('CSK Pair 2\n2020-09-26\n2020-11-13\nBperp: 623 m')
ax04.set_title('CSK Pair 3\n2020-09-27\n2021-08-12\nBperp: 1 m')
ax05.set_title('CSK Pair 4\n2021-02-17\n2021-02-18\nBperp: 89 m')
fig.subplots_adjust(hspace=0,wspace=0)

fig.savefig('/Users/markbemelmans/Documents/PhD/projects/Merapi2021/figures/Merapi_overview_v4.pdf')

In [16]:
%matplotlib osx
## overview figure

# 1   4   7  10  13 16 
# 2   5   8  11  14 17
# 3   6   9  12  15 18

# 1: True colour S2
# 2: CSK Amplitude
# 3: S1 Amplitude
# 4: S1 ifg
# 5: S1 range off
# 6: S1 azi off
# 7,10,13,16: CSK ifg
# 8,11,14,17: CSK range off
# 9,12,15,18: CSK azi off

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
# define map region of interest
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]

# zoom_lon_lims = [110.428, 110.45]
# zoom_lat_lims = [-7.545, -7.5305]
orientation = 'vertical'

zoom_lon_lims = [110.4387, 110.4495]
zoom_lat_lims = [-7.5411, -7.5344]

CSK_obj1 = example_pairs[0].Stack[1]
CSK_obj2 = example_pairs[1].Stack[1]
CSK_obj3 = example_pairs[2].Stack[1]
CSK_obj4 = example_pairs[3].Stack[1]


fig = plt.figure(figsize=(10,10))

gs = GridSpec(6, 6, figure=fig, height_ratios=[10,8,10,8,10,8])
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax20 = fig.add_subplot(gs[2, 0]) # CSK amplitude (with summit inset?)
ax40 = fig.add_subplot(gs[4, 0]) # S1 amplitude (with summit inset?)

ax01 = fig.add_subplot(gs[0, 1]) # S1 ifg
ax11 = fig.add_subplot(gs[1, 1]) # S1 ifg zoom
ax21 = fig.add_subplot(gs[2, 1]) # S1 range off
ax31 = fig.add_subplot(gs[3, 1]) # S1 range off zoom
ax41 = fig.add_subplot(gs[4, 1]) # S1 azi off
ax51 = fig.add_subplot(gs[5, 1]) # S1 azi off zoom

ax02 = fig.add_subplot(gs[0, 2]) # CSK1 ifg
ax12 = fig.add_subplot(gs[1, 2]) # CSK1 ifg zoom
ax22 = fig.add_subplot(gs[2, 2]) # CSK1 range off
ax32 = fig.add_subplot(gs[3, 2]) # CSK1 range off zoom
ax42 = fig.add_subplot(gs[4, 2]) # CSK1 azi off
ax52 = fig.add_subplot(gs[5, 2]) # CSK1 azi off

ax03 = fig.add_subplot(gs[0, 3]) # CSK2 ifg
ax13 = fig.add_subplot(gs[1, 3]) # CSK2 ifg zoom
ax23 = fig.add_subplot(gs[2, 3]) # CSK2 range off
ax33 = fig.add_subplot(gs[3, 3]) # CSK2 range off zoom
ax43 = fig.add_subplot(gs[4, 3]) # CSK2 azi off
ax53 = fig.add_subplot(gs[5, 3]) # CSK2 azi off zoom

ax04 = fig.add_subplot(gs[0, 4]) # CSK3 ifg
ax14 = fig.add_subplot(gs[1, 4]) # CSK3 ifg zoom
ax24 = fig.add_subplot(gs[2, 4]) # CSK3 range off
ax34 = fig.add_subplot(gs[3, 4]) # CSK3 range off zoom
ax44 = fig.add_subplot(gs[4, 4]) # CSK3 azi off
ax54 = fig.add_subplot(gs[5, 4]) # CSK3 azi off zoom

ax05 = fig.add_subplot(gs[0, 5]) # CSK4 ifg
ax15 = fig.add_subplot(gs[1, 5]) # CSK4 ifg zoom
ax25 = fig.add_subplot(gs[2, 5]) # CSK4 range off
ax35 = fig.add_subplot(gs[3, 5]) # CSK4 range off zoom
ax45 = fig.add_subplot(gs[4, 5]) # CSK4 azi off
ax55 = fig.add_subplot(gs[5, 5]) # CSK4 azi off zoom


# true colour
ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00.set_xlim(lon_lims)
ax00.set_ylim(lat_lims)
# ax00.scatter(x=[110.410626],y=[-7.526175],s=20,marker='s',color='tab:blue')
# ax00.scatter(x=[110.439757],y=[-7.537567],s=20,marker='v',color='tab:orange')
# ax00.scatter(x=[110.440165],y=[-7.537451],s=20,marker='v',color='tab:orange')
# ax00.scatter(x=[110.439260],y=[-7.537533],s=20,marker='v',color='tab:orange')

# ax00_1 = ax00.inset_axes([0.4,0.77,0.6,0.3])
# ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
# # world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
# ind_coastlines.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
# ax00_1.scatter(110.4,-7.54,s=5,c='red')
# ax00_1.set_aspect('equal','box')
# ax00_1.set_xlim((105,117))
# ax00_1.set_ylim((-9,-5))
# ax00_1.set_axis_off()

# S1 pwr
ax20.imshow(S1_PWR[470:675,1075:1335],cmap='Greys')

# CSK pwr
ax40.imshow(CSK_PWR[3400:5710,1440:3770],cmap='Greys')

# S1 ifg
ax01.imshow(S1_IFG,extent=S1_IFG_extent)
ax01.set_xlim(lon_lims)
ax01.set_ylim(lat_lims)
ax11.imshow(S1_IFG,extent=S1_IFG_extent)
ax11.set_xlim(zoom_lon_lims)
ax11.set_ylim(zoom_lat_lims)

# S1 r off
S1_r_off_map = ax21.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax21.set_xlim(lon_lims)
ax21.set_ylim(lat_lims)
S1_r_off_map_zoom = ax31.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax31.set_xlim(zoom_lon_lims)
ax31.set_ylim(zoom_lat_lims)

# S1 a off
S1_a_off_map = ax41.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax41.set_xlim(lon_lims)
ax41.set_ylim(lat_lims)
S1_a_off_map_zoom = ax51.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax51.set_xlim(zoom_lon_lims)
ax51.set_ylim(zoom_lat_lims)

# CSK ifg 1
ax02.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax02.set_xlim(lon_lims)
ax02.set_ylim(lat_lims)
ax12.imshow(CSK_IFG1,extent=CSK_IFG_extent1)
ax12.set_xlim(zoom_lon_lims)
ax12.set_ylim(zoom_lat_lims)

# CSK r off 1
CSK_r_off_map = ax22.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax22.set_xlim(lon_lims)
ax22.set_ylim(lat_lims)
CSK_r_off_map_zoom = ax32.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax32.set_xlim(zoom_lon_lims)
ax32.set_ylim(zoom_lat_lims)
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 1
CSK_a_off_map = ax42.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
CSK_a_off_map_zoom = ax52.hexbin(CSK_obj1.Lon_off.flatten(),CSK_obj1.Lat_off.flatten(),C=CSK_obj1.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 2
ax03.imshow(CSK_IFG2,extent=CSK_IFG_extent2)
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
ax13.imshow(CSK_IFG2,extent=CSK_IFG_extent2)
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)

# CSK r off 2
CSK_r_off_map = ax23.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
CSK_r_off_map = ax33.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 2
CSK_a_off_map = ax43.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax42.set_xlim(lon_lims)
ax42.set_ylim(lat_lims)
CSK_a_off_map = ax53.hexbin(CSK_obj2.Lon_off.flatten(),CSK_obj2.Lat_off.flatten(),C=CSK_obj2.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax52.set_xlim(zoom_lon_lims)
ax52.set_ylim(zoom_lat_lims)
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 3
ax04.imshow(CSK_IFG3,extent=CSK_IFG_extent3)
ax04.set_xlim(lon_lims)
ax04.set_ylim(lat_lims)
ax14.imshow(CSK_IFG3,extent=CSK_IFG_extent3)
ax14.set_xlim(zoom_lon_lims)
ax14.set_ylim(zoom_lat_lims)

# CSK r off 3
CSK_r_off_map = ax24.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax24.set_xlim(lon_lims)
ax24.set_ylim(lat_lims)
CSK_r_off_map = ax34.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax34.set_xlim(zoom_lon_lims)
ax34.set_ylim(zoom_lat_lims)

# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 3
CSK_a_off_map = ax44.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax44.set_xlim(lon_lims)
ax44.set_ylim(lat_lims)
CSK_a_off_map = ax54.hexbin(CSK_obj3.Lon_off.flatten(),CSK_obj3.Lat_off.flatten(),C=CSK_obj3.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-15,vmax=15)
ax54.set_xlim(zoom_lon_lims)
ax54.set_ylim(zoom_lat_lims)

# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


# CSK ifg 4
ax05.imshow(CSK_IFG4,extent=CSK_IFG_extent4)
ax05.set_xlim(lon_lims)
ax05.set_ylim(lat_lims)
ax15.imshow(CSK_IFG4,extent=CSK_IFG_extent4)
ax15.set_xlim(zoom_lon_lims)
ax15.set_ylim(zoom_lat_lims)

# CSK r off 4
CSK_r_off_map = ax25.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax25.set_xlim(lon_lims)
ax25.set_ylim(lat_lims)
CSK_r_off_map = ax35.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax35.set_xlim(zoom_lon_lims)
ax35.set_ylim(zoom_lat_lims)
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# CSK a off 4
CSK_a_off_map = ax45.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax45.set_xlim(lon_lims)
ax45.set_ylim(lat_lims)
CSK_a_off_map = ax55.hexbin(CSK_obj4.Lon_off.flatten(),CSK_obj4.Lat_off.flatten(),C=CSK_obj4.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=-1,vmax=1)
ax55.set_xlim(zoom_lon_lims)
ax55.set_ylim(zoom_lat_lims)
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)

for a in [ax00,ax01,ax21,ax41,ax02,ax22,ax42,ax03,ax23,ax43,ax04,ax24,ax44,ax05,ax25,ax45]:
    a.set_aspect('equal','box')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.plot([zoom_lon_lims[0],zoom_lon_lims[1],zoom_lon_lims[1],zoom_lon_lims[0],zoom_lon_lims[0]],
           [zoom_lat_lims[0],zoom_lat_lims[0],zoom_lat_lims[1],zoom_lat_lims[1],zoom_lat_lims[0]],
           linewidth=2,color='red')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
        "family": "serif",
        "size": "large",
    }))

for a in [ax11,ax31,ax51,ax12,ax32,ax52,ax13,ax33,ax53,ax14,ax34,ax54,ax15,ax35,ax55]:
    a.set_aspect('equal','box')
    
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    # a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left',font_properties={
    #     "family": "serif",
    #     "size": "large",
    # }))
    a.set_xlim(zoom_lon_lims)
    a.set_ylim(zoom_lat_lims)
    a.set_axis_off()
    

for (a,s) in zip([ax00,ax01,ax21,ax41,ax02,ax22,ax42,ax03,ax23,ax43,ax04,ax24,ax44,ax05,ax25,ax45],['A','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R']):
    a.annotate(s,xy=[lon_lims[0]+0.002,lat_lims[0]+0.002],xytext=[lon_lims[0]+0.002,lat_lims[0]+0.002],xycoords='data',backgroundcolor='white',fontsize=16)
    a.set_axis_off()
ax20.annotate('B', xy=[0.01,0.01],xytext=[0.01,0.01],xycoords='axes fraction',backgroundcolor='white',fontsize=16)
ax20.set_axis_off()
ax40.annotate('C', xy=[0.01,0.01],xytext=[0.01,0.01],xycoords='axes fraction',backgroundcolor='white',fontsize=16)
ax40.set_axis_off()

ax05_cmap = fig.add_axes([0.9,0.67,0.01,0.33])
ax05_cmap.set_axis_off()

fake_data = ax05.imshow(np.random.random((10,2)),cmap=colormap, vmin=-1,vmax=1,alpha = 1)



# ax05_cax = add_right_cax(ax05,pad=0.01,width=0.01)
# cbar0 = fig.colorbar(fake_data, cax=ax05_cax, orientation='vertical',)
# ticklabs = cbar0.ax.get_yticklabels()
# cbar0.ax.set_yticklabels(ticklabs, fontsize=16)
# cbar0.set_label( label='LOS change', fontsize=16)
# cbar0.set_ticks(ticks=[-1, 0, 1], labels=[r'-λ', '0', r'+λ'])
# ax15_cax = add_right_cax(ax15,pad=0.01,width=0.01)
# cbar1 = fig.colorbar(CSK_r_off_map, cax=ax15_cax, orientation='vertical', )
# ticklabs = cbar1.ax.get_yticklabels()
# cbar1.ax.set_yticklabels(ticklabs,fontsize=16)
# cbar1.set_label(label='Range offset [m]', size=16)
# cbar1.set_ticks(ticks=[-3, 0, 3],labels=['-3','0','3'])
# ax25_cax = add_right_cax(ax25,pad=0.01,width=0.01)
# cbar2 = fig.colorbar(CSK_a_off_map, cax=ax25_cax, orientation='vertical')
# ticklabs = cbar2.ax.get_yticklabels()
# print(ticklabs)
# cbar2.ax.set_yticklabels(ticklabs,fontsize=16)
# cbar2.set_label(label= 'Azimuth offset [m]',size=16)
# cbar2.set_ticks(ticks=[-3, 0, 3],labels=['-3','0','3'])



# set titles
ax01.set_title('Sentinel-1\nBperp: 11 m ')
ax02.set_title('CSK Pair 1\nBperp: 4 m')
ax03.set_title('CSK Pair 2\nBperp: 623 m')
ax04.set_title('CSK Pair 3\nBperp: 1 m')
ax05.set_title('CSK Pair 4\nBperp: 89 m')
fig.subplots_adjust(hspace=0,wspace=0)


: 

In [16]:
plt.close('all')

In [27]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

from matplotlib.colors import LinearSegmentedColormap
clist = ['#f579cd', '#f67fc6', '#f686bf', '#f68cb9', '#f692b3', '#f698ad',
                     '#f69ea7', '#f6a5a1', '#f6ab9a', '#f6b194', '#f6b78e', '#f6bd88',
                     '#f6c482', '#f6ca7b', '#f6d075', '#f6d66f', '#f6dc69', '#f6e363',
                     '#efe765', '#e5eb6b', '#dbf071', '#d0f477', '#c8f67d', '#c2f684',
                     '#bbf68a', '#b5f690', '#aff696', '#a9f69c', '#a3f6a3', '#9cf6a9',
                     '#96f6af', '#90f6b5', '#8af6bb', '#84f6c2', '#7df6c8', '#77f6ce',
                     '#71f6d4', '#6bf6da', '#65f6e0', '#5ef6e7', '#58f0ed', '#52e8f3',
                     '#4cdbf9', '#7bccf6', '#82c4f6', '#88bdf6', '#8eb7f6', '#94b1f6',
                     '#9aabf6', '#a1a5f6', '#a79ef6', '#ad98f6', '#b392f6', '#b98cf6',
                     '#bf86f6', '#c67ff6', '#cc79f6', '#d273f6', '#d86df6', '#de67f6',
                     '#e561f6', '#e967ec', '#ed6de2', '#f173d7']
colormap = LinearSegmentedColormap.from_list('dismph', clist, N=256)
#colormap = self.cmap_map(lambda x: x/2 + 0.5, colormap)  # brighten colormap
#colormap = self.cmap_map(lambda x: x*0.75, colormap)     # darken colormap
colormap.set_bad('w', 0.0)

dismph = colormap(range(255))
dismph2 = np.row_stack((np.array(dismph),np.array(dismph)))
print(dismph2)

In [9]:
## overview figure

# 1   2   3  
# 4   5   6   
# 7   8   9   

# 1: True colour S2
# 2: CSK Amplitude
# 3: S1 Amplitude
# 4: CSK ifg
# 5: CSK range off
# 6: CSK azi off
# 7: S1 ifg
# 8: S1 range off
# 9: S1 azi off

%matplotlib osx

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

CSK_obj = example_pairs[0].Stack[1]

fig = plt.figure()

gs = GridSpec(3, 3, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax01 = fig.add_subplot(gs[0, 1]) # CSK amplitude (with summit inset?)
ax02 = fig.add_subplot(gs[0, 2]) # S1 amplitude (with summit inset?)
ax10 = fig.add_subplot(gs[1, 0]) # CSK ifg
ax11 = fig.add_subplot(gs[1, 1]) # CSK range off
ax12 = fig.add_subplot(gs[1, 2]) # CSK azi off
ax20 = fig.add_subplot(gs[2, 0]) # S1 ifg
ax21 = fig.add_subplot(gs[2, 1]) # S1 range off
ax22 = fig.add_subplot(gs[2, 2]) # S1 azi off



ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00_1 = ax00.inset_axes([0.5,0.75,0.5,0.25])
ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
# world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ind_coastlines.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ax00_1.scatter(110.4,-7.54,s=5,c='red')
ax00_1.set_aspect('equal','box')
ax00_1.set_xlim((105,117))
ax00_1.set_ylim((-9,-5))
ax00_1.set_axis_off()


# S1_PWR_map = ax01.imshow(S1_PWR[250:1000,750:2000],cmap='Greys')
# ax01_1 = inset_axes(ax01,width='50%',height='50%',loc='upper right')
# ax01_1.imshow(S1_PWR[520:625,1100:1310],cmap='Greys')
ax01.imshow(S1_PWR[470:675,1075:1335],cmap='Greys')


# CSK_PWR_map = ax02.imshow(CSK_PWR,cmap='Greys')
# ax02_1 = inset_axes(ax02,width='50%',height='50%',loc='upper right')
# ax02_1.imshow(CSK_PWR[3900:5210,1940:3270],cmap='Greys')
ax02.imshow(CSK_PWR[3400:5710,1440:3770],cmap='Greys')
# ax02_1 = inset_axes(ax02,width='50%',height='50%',loc='lower left')
# ax02_1.imshow(CSK_PWR[4100:4300,2190:2390],cmap='Greys')

ax10.imshow(S1_IFG,extent=S1_IFG_extent)
S1_r_off_map = ax11.hexbin(S1_data[10],S1_data[9],C=S1_data[7],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
S1_a_off_map = ax12.hexbin(S1_data[10],S1_data[9],C=S1_data[8],gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])

ax20.imshow(CSK_IFG,extent=CSK_IFG_extent)

CSK_r_off_map = ax21.hexbin(CSK_obj.Lon_off.flatten(),CSK_obj.Lat_off.flatten(),C=CSK_obj.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

CSK_a_off_map = ax22.hexbin(CSK_obj.Lon_off.flatten(),CSK_obj.Lat_off.flatten(),C=CSK_obj.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
# plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)


for a in [ax00,ax10,ax11,ax12,ax20,ax21,ax22]:
    a.set_aspect('equal','box')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper left'))

for a,s in zip([ax00,ax01,ax02,ax10,ax11,ax12,ax20,ax21,ax22],['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R']):
    a.set_axis_off()
    a.annotate(s,xy=[0,0],xytext=[0,0],xycoords='figure fraction',)



add_right_cax(ax05)

NameError: name 'CSK_IFG' is not defined

Traceback (most recent call last):
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backend_bases.py", line 1226, in _on_timer
    ret = func(*args, **kwargs)
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_macosx.py", line 68, in callback_func
    callback()
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_macosx.py", line 88, in _draw_idle
    self.draw()
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_macosx.py", line 50, in draw
    super().draw()
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py", line 400, in draw
    self.figure.draw(self.renderer)
  File "/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/matplotlib/artist.py", line 95, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "/Applications/anaconda3/envs/

In [2]:
import mintpy


from mintpy.colors import ColormapExt
mph = ColormapExt('dismph').colormap
print(mph)

ModuleNotFoundError: No module named 'mintpy.colors'

In [ ]:
plt.close('all')

In [10]:
%matplotlib osx

from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# make figure
grid_size = 1000
cmap=cm.vik
clims = [-3, 3]
lon_lims = [110.425, 110.46]
lat_lims = [-7.555, -7.515]
orientation = 'vertical'

obj = example_pairs[0].Stack[1]

fig = plt.figure()

gs = GridSpec(2, 4, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # Merapi true colour with inset
ax10 = fig.add_subplot(gs[1, 0]) # Merapi NDVI

ax01 = fig.add_subplot(gs[0, 1]) # Merapi elevation
ax11 = fig.add_subplot(gs[1, 1]) # Merapi slope
ax02 = fig.add_subplot(gs[0, 2]) # Merapi shadow + overlay
ax12 = fig.add_subplot(gs[1, 2]) # Merapi ifg
ax03 = fig.add_subplot(gs[0, 3]) # range offset
ax13 = fig.add_subplot(gs[1, 3]) # azimuth offset


ax00.imshow(TRUE_COL,extent=TRUE_COL_extent)
ax00_1 = inset_axes(ax00,width='50%',height='25%',loc=2)
ax00_1.add_patch(Rectangle((90,-15),55,25,color='white'))
world.plot(ax=ax00_1,color='lightgrey',edgecolor='black')
ax00_1.scatter(110.4,-7.54,s=5,c='red')
ax00_1.set_aspect('equal','box')
ax00_1.set_xlim((90,145))
ax00_1.set_ylim((-15,10))
ax00_1.set_axis_off()

ndvi_map = ax10.imshow(NDVI,cmap=ndvi_cmap,extent=NDVI_extent,vmin=-1,vmax=1)
plt.colorbar(ndvi_map,label='NDVI [-]',orientation=orientation)

dem_map = ax01.imshow(DEM_heights,cmap='gist_earth',extent=DEM_extent,vmin=0,vmax=3000)
ax01.imshow(SHADING, cmap=cm.grayC, alpha=0.3 , extent=DEM_extent)
plt.colorbar(dem_map,label='elevation [m]',orientation=orientation)

slope_map = ax11.imshow(DEM_slope,cmap='Greys_r',extent=DEM_extent,vmin=0,vmax=45)
plt.colorbar(slope_map,label='slope [deg.]', extend='max',orientation=orientation)

ax02.imshow(ls_map,cmap='Greys',extent=DEM_extent)
ax02.scatter([],[],s=30,c='black',edgecolor='black',label='Shadow')
ax02.scatter([],[],s=30,c='white',edgecolor='black',label='Overlay')
ax02.scatter([],[],s=30,c='grey',edgecolor='black',label='Normal or\nforeshortening')
ax02.legend(loc='best')

ax12.imshow(IFG,extent=IFG_extent)


# ax03.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_extent)
r_off_map = ax03.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
plt.colorbar(r_off_map,label='range offset [m]',orientation=orientation)

# ax13.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_extent)
a_off_map = ax13.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
plt.colorbar(a_off_map,label='azimuth offset [m]',orientation=orientation)

print(lon_lims,lat_lims)

for (ax,text) in zip([ax00,ax10,ax01,ax11,ax02,ax12,ax03,ax13],['A','B','C','D','E','F','G','H']):
    ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='upper right'))
    if ax ==ax00:
        ax.text(110.44570,-7.52014,text,fontsize=20,backgroundcolor=(1,1,1,0.3))
    else:
        ax.text(110.42633,-7.52014,text,fontsize=20,backgroundcolor=(1,1,1,0.3))
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_axis_off()

# fig.tight_layout()

[110.425, 110.46] [-7.555, -7.515]


2023-09-21 16:06:00.545 python[86118:1940784] +[CATransaction synchronize] called within transaction
2023-09-21 16:06:07.086 python[86118:1940784] +[CATransaction synchronize] called within transaction
2023-09-21 16:06:08.220 python[86118:1940784] +[CATransaction synchronize] called within transaction


In [7]:
# plt.close('all')